# Unit Cube

This Nootebook have the goal to calculate the total energy inside a hexahedric element
applying the continum mechanics equations [Bonet 1997]

In [50]:
# Libraries 
import numpy as np
import jax.numpy as jnp
import matplotlib.pyplot as plt
import pyvista as pv
from jax import random
import jax
key = random.key(0)

In [51]:
## import mesh

mesh = pv.read('/home/nicolas/Escritorio/Codigos/JAX/Unit_Tests/unit_cube.msh')  #Orgiinal File
mesh.clear_data()

## displacement from FEM software Vulcan
disp = np.load('/home/nicolas/Escritorio/Codigos/JAX/Unit_Tests/cube_displacements.npy')[-1,:,:]  # The orden of this file is [time, node, dim]

mesh_def = mesh.copy()
mesh_def.points += disp

pl = pv.Plotter(shape=(1, 2))


pl.subplot(0, 0)
pl.add_text("Original State", font_size=30)
pl.add_mesh(mesh, show_edges=True, color='lightblue',opacity = 1)


pl.subplot(0, 1)
pl.add_text("Deformed State", font_size=30)
pl.add_mesh(mesh_def, show_edges=True, color='lightblue')


# Display the window
pl.show()

Widget(value="<iframe src='http://localhost:44449/index.html?ui=P_0x7fa220679d20_4&reconnect=auto' style='widt…

Work line is:

displacement -> deformation gradient -> Energy -> Total Energy Integral



In [98]:
class energy():

    def __init__(self, nodes, conn):
        self.conn = conn
        self.nodes = nodes[conn]
        self.nnodes = 8


        puntos_iso = jnp.array([[-1,-1,-1],
                               [ 1,-1,-1],
                               [ 1, 1,-1],
                               [-1, 1,-1],
                               [-1,-1, 1],
                               [ 1,-1, 1],
                               [ 1, 1, 1],
                               [-1, 1, 1] ])
        

        self.gauss_points = jnp.array([[-1/3**0.5, -1/3**0.5, -1/3**0.5],
                                      [ 1/3**0.5, -1/3**0.5, -1/3**0.5],
                                      [ 1/3**0.5,  1/3**0.5, -1/3**0.5],
                                      [-1/3**0.5,  1/3**0.5, -1/3**0.5],
                                      [-1/3**0.5, -1/3**0.5,  1/3**0.5],
                                      [ 1/3**0.5, -1/3**0.5,  1/3**0.5],
                                      [ 1/3**0.5,  1/3**0.5,  1/3**0.5],
                                      [-1/3**0.5,  1/3**0.5,  1/3**0.5],
                                      ])
        
        self.der_N_X_gp = [self.der_N_X(i) for i in self.gauss_points]

        self.der_N_X_gp = jnp.array(self.der_N_X_gp).transpose((1,0,2,3))

    def _get_nodes(self, x):
        return x[self.conn,:]


    def N_func(self, xi):
        xi0 = xi[0]
        xi1 = xi[1]
        xi2 = xi[2]
        N1 = (1.0 - xi0)*(1.0 - xi1)*(1.0 - xi2)/8.0  
        N2 = (1.0 + xi0)*(1.0 - xi1)*(1.0 - xi2)/8.0  
        N3 = (1.0 + xi0)*(1.0 + xi1)*(1.0 - xi2)/8.0  
        N4 = (1.0 - xi0)*(1.0 + xi1)*(1.0 - xi2)/8.0  
        N5 = (1.0 - xi0)*(1.0 - xi1)*(1.0 + xi2)/8.0  
        N6 = (1.0 + xi0)*(1.0 - xi1)*(1.0 + xi2)/8.0  
        N7 = (1.0 + xi0)*(1.0 + xi1)*(1.0 + xi2)/8.0
        N8 = (1.0 - xi0)*(1.0 + xi1)*(1.0 + xi2)/8.0
        return jnp.array([N1, N2, N3, N4, N5, N6, N7, N8])

    def der_N_fun(self, xi):
        xi0 = xi[0]
        xi1 = xi[1]
        xi2 = xi[2]
        return jnp.array([[  -(1.0 - xi1)*(1.0 - xi2)/8.0, -(1.0 - xi0)*(1.0 - xi2)/8.0, -(1.0 - xi0)*(1.0 - xi1)/8.0],
                         [   (1.0 - xi1)*(1.0 - xi2)/8.0, -(1.0 + xi0)*(1.0 - xi2)/8.0, -(1.0 + xi0)*(1.0 - xi1)/8.0],
                         [   (1.0 + xi1)*(1.0 - xi2)/8.0,  (1.0 + xi0)*(1.0 - xi2)/8.0, -(1.0 + xi0)*(1.0 + xi1)/8.0],
                         [  -(1.0 + xi1)*(1.0 - xi2)/8.0,  (1.0 - xi0)*(1.0 - xi2)/8.0, -(1.0 - xi0)*(1.0 + xi1)/8.0],
                         [  -(1.0 - xi1)*(1.0 + xi2)/8.0, -(1.0 - xi0)*(1.0 + xi2)/8.0,  (1.0 - xi0)*(1.0 - xi1)/8.0],
                         [   (1.0 - xi1)*(1.0 + xi2)/8.0, -(1.0 + xi0)*(1.0 + xi2)/8.0,  (1.0 + xi0)*(1.0 - xi1)/8.0],
                         [   (1.0 + xi1)*(1.0 + xi2)/8.0,  (1.0 + xi0)*(1.0 + xi2)/8.0,  (1.0 + xi0)*(1.0 + xi1)/8.0],
                         [  -(1.0 + xi1)*(1.0 + xi2)/8.0,  (1.0 - xi0)*(1.0 + xi2)/8.0,  (1.0 - xi0)*(1.0 + xi1)/8.0],
                         ]) 

    def der_X_xi(self, xi):  # 7.6b
        return jnp.einsum('...ai,aj', self.nodes, jax.jit(self.der_N_fun)(xi))

    def der_N_X(self, xi):  # 7.6b
        temp = self.der_X_xi(xi).transpose(0,2,1)
        inv_der_X_xi = jnp.linalg.inv(temp)
        out = jnp.matmul(inv_der_X_xi, jax.jit(self.der_N_fun)(xi).T).transpose(0,2,1)
        ##print(out.shape)
        return out

    def der_x_xi(self, x, xi):  # 7.11a
        return jnp.einsum('...ai,...aj', x, jax.jit(self.der_N_fun)(xi))

    def der_N_x(self, x, xi):  # 7.11b
        temp = self.der_x_xi(x, xi).transpose(0,2,1)
        inv_der_x_xi = jnp.linalg.inv(temp)
        
        return jnp.matmul(inv_der_x_xi,jax.jit(self.der_N_fun)(xi).T).transpose(0,2,1)
    
    def f_gauss(self, x_n):  # gradiente de deformacion -- 7.5
        #print("disp", x_n)
        x = self._get_nodes(x_n)
        #print(f"xn: {x}")
        #print(self.der_N_X_gp)
        F = jnp.einsum('eai,exaj->exij', x, self.der_N_X_gp)
        #print("test", F)
        return F
    
    def Delphino(self,F,c1,c2):
        F_T = jnp.moveaxis(F,-1,-2)
        C = jnp.matmul(F_T,F)
        I1,I2,I3 = jnp.einsum('...ii',C), 0.5 * (jnp.einsum('...ii',C)**2 - jnp.einsum('...ii',jnp.matmul(C,C))), jnp.linalg.det(C)
        aux = I3**(-1/3)
        energia  =  (c1/c2) * (jnp.e**(c2*0.5*(I1*aux-3)) - 1 )    
        #print(energia)    
        return energia


    def Delphino_E(self,x_n,c1,c2):
        # x_n nuevas coordenadas
        #print(x_n)
        x = jax.jit(self._get_nodes)(x_n)
        F = jax.jit(self.f_gauss)(x_n)
        # print(F.shape)
        temp = self.Delphino(F,c1,c2)
        # print(temp.shape)
        micro = []
        for it, gp in enumerate(self.gauss_points):
            #print(x.shape)
            aux = jnp.linalg.det(jax.jit(self.der_x_xi)(x, gp))
            micro.append(aux)
        e_t = jnp.dot(temp,jnp.array(micro))
        # print(jnp.array(micro).sum())
        # # print(e_t)
        # #print(micro)
        # print(x_n)
        return e_t

In [99]:
Elemento = energy(mesh.points, mesh.cells_dict[12])

In [100]:
import time


start = time.time()
Elemento.Delphino_E(mesh.points, 30.0E-3,3.77)
end = time.time()
print('Tiempo Primera Ejecucion',end - start)



Tiempo Primera Ejecucion 1.2261955738067627


In [101]:
for i in range(5):
    start = time.time()
    print(Elemento.Delphino_E(mesh_def.points, 30.0E-3,3.77))
    end = time.time()
    print('Tiempo Compilado',end - start)

[[7.9306474]]
Tiempo Compilado 1.1379156112670898
[[7.9306474]]
Tiempo Compilado 1.1092591285705566
[[7.9306474]]
Tiempo Compilado 1.1071689128875732
[[7.9306474]]
Tiempo Compilado 1.0973515510559082
[[7.9306474]]
Tiempo Compilado 1.2464449405670166
